In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

object_sizes = [128, 256, 512, 1024]
num_clients = [1, 5, 10, 100]
num_samples = [100, 200, 400, 800]

test_types = ['put', 'get', 'delete']

def get_name(test_type, object_size, num_client, num_sample):
    return test_type + '_' + str(object_size) + '_' + str(num_client) + '_' + str(num_sample) + '.csv'

csv_names = [get_name(test_type, object_size, num_client, num_sample)
    for object_size in object_sizes
    for num_client in num_clients
    for num_sample in num_samples]

print(len(csv_names))
print(csv_names)

all_latency = {csv_name: pd.read_csv(csv_name)['latency'].apply(pd.to_numeric).values for csv_name in csv_names}

In [ ]:
import statsmodels.api as sm

for test_type in test_types:
    name = get_name('put', 4096, 2, 500)
    latency = all_latency[name]

    plt.suptitle(name)

    plt.subplot(221)
    plt.plot(latency)
    plt.ylabel('Latency')

    plt.subplot(222)
    plt.plot(sorted(latency))

    plt.subplot(212)
    # Create the ECDF
    ecdf = sm.distributions.ECDF(latency)
    # Generate x values for plotting
    x = np.linspace(min(latency), max(latency), num=len(latency))
    # Calculate y values using the ECDF
    y = ecdf(x)
    # Plot the ECDF
    plt.grid()
    plt.step(x, y, where='post')
    plt.xlabel('Latency')
    plt.ylabel('ECDF')
    plt.title('Empirical Cumulative Distribution Function')
    plt.tight_layout()
    plt.show()

In [ ]:
curves = {}
for name in csv_names:
    t = name.split('_')
    if t[0] == 'put':
        latency = sorted(all_latency[name])
        ecdf = sm.distributions.ECDF(latency)
        x = np.linspace(min(latency), max(latency), num=len(latency))
        y = ecdf(x)
        curves[name] = {'x': x, 'y': y}

# Create a figure and axis
fig, ax = plt.subplots()

# Plot the curves and add names
for name, curve_data in curves.items():
    x = curve_data['x']
    y = curve_data['y']
    ax.plot(x, y, label=name)

# Add legend and labels
ax.grid()
ax.legend()
plt.xlabel('Latency')
plt.ylabel('ECDF')
plt.title('Empirical Cumulative Distribution Function')


image_format = 'svg' # e.g .png, .svg, etc.
image_name = 'ECDF.svg'

fig.savefig(image_name, format=image_format, dpi=1200)
